# CE-QUAL-W2 Parser

In [1]:
import pandas as pd
import numpy as np
from w2_control_file import W2ControlFile
import w2_functions as w2

## Load the CE-QUAL-W2 control file

In [2]:
infile = 'w2_con.npt'
w2con = W2ControlFile(infile)
# w2con.save()

## Read the card names

In [3]:
card_names = []

with open('w2_card_names.txt', 'r') as f:
	for line in f.readlines():
		card_names.append(line.strip())

print(card_names)

['GRID', 'IN/OUTFL', 'CONSTITU', 'MISCELL', 'TIME CON', 'DLT CON', 'DLT DATE', 'DLT MAX', 'DLT FRN', 'DLT LIMIT', 'BRANCH G', 'LOCATION', 'INIT CND', 'CALCULAT', 'DEAD SEA', 'INTERPOL', 'HEAT EXCH', 'ICE COVER', 'TRANSPORT', 'HYD COEF', 'EDDY VISC', 'N STRUC', 'STR INT', 'STR TOP', 'STR BOT', 'STR SINK', 'STR ELEV', 'STR WIDTH', 'PIPES', 'PIPE UP', 'PIPE DOWN', 'SPILLWAYS', 'SPILL UP', 'SPILL DOWN', 'SPILL GAS', 'GATES', 'GATE WEIR', 'GATE UP', 'GATE DOWN', 'GATE GAS', 'PUMPS 1', 'PUMPS 2', 'WEIR SEG', 'WEIR TOP', 'WEIR BOT', 'WD INT', 'WD SEG', 'WD ELEV', 'WD TOP', 'WD BOT', 'TRIB PLA', 'TRIB INT', 'TRIB SEG', 'TRIB TOP', 'TRIB BOT', 'DST TRIB', 'HYD PRINT', 'SNP PRINT', 'SNP DATE', 'SNP FREQ', 'SNP SEG', 'SCR PRINT', 'SCR DATE', 'SCR FREQ', 'PRF PLOT', 'PRF DATE', 'PRF FREQ', 'PRF SEG', 'SPR PLOT', 'SPR DATE', 'SPR FREQ', 'SPR SEG', 'VPL PLOT', 'VPL DATE', 'VPL FREQ', 'CPL PLOT', 'CPL DATE', 'CPL FREQ', 'FLUXES', 'FLX DATE', 'FLX FREQ', 'TSR PLOT', 'TSR DATE', 'TSR FREQ', 'TSR SEG', 

In [4]:
card_locations = pd.DataFrame(np.zeros(len(card_names), dtype=int), index=card_names, columns=['line_start'])

for card_name in card_names:
	for i, line in enumerate(w2con.lines):
		if line.upper().startswith(card_name):
			card_locations.loc[card_name, 'line_start'] = i

# pd.set_option('display.max_rows', None)

## Compute the number of lines between cards

In [5]:
diffs = card_locations.diff(periods=-1) * -1
diffs.iloc[-1,0] = 3 # The diffs fall one record short. The last several records are always three lines.
card_locations['num_lines'] = diffs
convert_dict = {'num_lines': int}
card_locations = card_locations.astype(convert_dict)

## Read all the cards into data frames

In [17]:
widths = 10*[8]
nlines = len(w2con.lines)

cards = []
for card_name in card_names:
	record_line = card_locations.loc[card_name, 'line_start'] + 1 # account for get_record using a start index of 1 for the lines instead of zero.
	record_nlines = card_locations.loc[card_name, 'num_lines'] - 1
	data = w2.get_record(w2con.w2_control_filepath, widths, nlines, record_line, record_nlines)
	cards.append(data)

## Write the cards to Excel

In [102]:
def update_format(curr_frmt, new_prprty, wrkbk):
    """
    Update a cell's existing format with new properties
    """
    new_frmt = curr_frmt.__dict__.copy()

    for k, v in new_prprty.items():
        new_frmt[k] = v
    new_frmt = { k: v for k, v in new_frmt.items() if (v != 0) and (v is not None) and (v != {}) and (k != 'escapes') }

    print(new_frmt)

    return wrkbk.add_format(new_frmt)

# Put multiple data frames in an Excel sheet
def multiple_dfs(df_list, sheets, file_name, spaces):
    writer = pd.ExcelWriter(file_name, engine='xlsxwriter')   

    row = 0
    for dataframe in df_list:
        dataframe.to_excel(writer, sheet_name=sheets, startrow=row, startcol=0)   
        row = row + len(dataframe.index) + spaces + 1

    # Change the default formatting to remove the borders
    workbook = writer.book
    worksheet = writer.sheets['w2_con']
    
    # Mimic the default pandas header format for use later
    hdr_fmt = workbook.add_format({
        'bold': False,
        'border': 0,
        'text_wrap': True,
        'align': 'center'
    })

    new_fmt = update_format(hdr_fmt, {
        'bold': False,
        'border': 0,
        'border_color': 'black', # Setting the border color is the only option that turned the borders off (when border is set to 0).
        'align': 'left',
        'text_h_align': 1, # left
        'locked': False
        }, workbook)

    for i in range(15):
        for j in range(2000):
            worksheet.conditional_format(j, i, j, i, {
                'type': 'formula',
                'criteria': 'True',
                'format': new_fmt
            })

    writer.save()

In [103]:
multiple_dfs(cards, 'w2_con', 'cards.xlsx', 1)	

{'num_format': 'General', 'font_name': 'Calibri', 'font_size': 11, 'font_family': 2, 'font_scheme': 'minor', 'text_h_align': 1, 'text_wrap': True, 'border_color': 'black', 'align': 'left'}
